In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

# read country_profile_variables.csv and keep interesting features

df_profile = pd.read_csv('/kaggle/input/undata-country-profiles/country_profile_variables.csv', index_col='country')
df_profile = df_profile[[
            'Population in thousands (2017)',
            'GDP per capita (current US$)',
            'Unemployment (% of labour force)',
            'Urban population (% of total population)',
            'Infant mortality rate (per 1000 live births',
            'Health: Total expenditure (% of GDP)',
            'Education: Government expenditure (% of GDP)',
            'Education: Primary gross enrol. ratio (f/m per 100 pop.)',
            'Education: Secondary gross enrol. ratio (f/m per 100 pop.)',
            'Education: Tertiary gross enrol. ratio (f/m per 100 pop.)',
            'Pop. using improved drinking water (urban/rural, %)'
]]

In [ ]:
# rename some columns
df_profile.rename(index={'United States of America':'United States'}, inplace=True)
df_profile.rename(columns={
            'Population in thousands (2017)':'population',
            'GDP per capita (current US$)':'GDP_capita',
            'Unemployment (% of labour force)':'unempl',
            'Urban population (% of total population)':'urban',
            'Infant mortality rate (per 1000 live births':'inf_mort',
            'Health: Total expenditure (% of GDP)':'health_exp',
            'Education: Government expenditure (% of GDP)':'ed_exp',
            'Education: Primary gross enrol. ratio (f/m per 100 pop.)':'ed_prim_enrol',
            'Education: Secondary gross enrol. ratio (f/m per 100 pop.)':'ed_sec_enrol',
            'Education: Tertiary gross enrol. ratio (f/m per 100 pop.)':'ed_ter_enrol',
            'Pop. using improved drinking water (urban/rural, %)':'impr_water'
}, inplace=True)

In [ ]:
# read 'country_vaccinations.csv' and concatenate with country profiles

df_vac = pd.read_csv('/kaggle/input/covid-world-vaccination-progress/country_vaccinations.csv')
df_vac = df_vac[df_vac['date']=='2021-01-21'].set_index('country')
df_vac = pd.concat([df_vac,df_profile.reindex(df_vac.index)], axis=1)
print(df_vac.columns)

In [ ]:
# drop columns 
df_vac.drop(['iso_code', 'date', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'daily_vaccinations_raw',
       'people_vaccinated_per_hundred', 'people_fully_vaccinated_per_hundred', 'vaccines', 'source_name', 'source_website'], axis=1, inplace=True)
#df_vac.drop(['Isle of Man'], axis=0, inplace=True)

In [ ]:
# two functions for transforming string data to float
def average(string):
    sum = 0
    try:
        list = string.split('/')
        for n in list:
            sum = sum + float(n)
        return sum/len(list)
    except:
        return -99
def to_float(string):
    try:
        num = float(string)
        return num
    except:
        return -99

In [ ]:
df_vac['ed_prim_enrol'] = df_vac['ed_prim_enrol'].apply(average)
df_vac['ed_sec_enrol'] = df_vac['ed_sec_enrol'].apply(average)
df_vac['ed_ter_enrol'] = df_vac['ed_ter_enrol'].apply(average)
df_vac['impr_water'] = df_vac['impr_water'].apply(average)
df_vac['unempl'] = df_vac['unempl'].apply(to_float)
df_vac['inf_mort'] = df_vac['inf_mort'].apply(to_float)
df_vac['ed_exp'] = df_vac['ed_exp'].apply(to_float)
print(df_vac.info())

In [ ]:
df_vac_corr = df_vac.corr().tail(11)
print(df_vac_corr)

In [ ]:
# correlation of 'daily_vaccinations'
df_corr_daily_vac = df_vac_corr['daily_vaccinations'].sort_values()
print(df_corr_daily_vac)
df_corr_daily_vac.plot.bar()

In [ ]:
# plot correlation of "total_vaccinations_per_hundred"
df_corr_totalvac_per_hun = df_vac_corr['total_vaccinations_per_hundred'].sort_values()
print(df_corr_totalvac_per_hun)
df_corr_totalvac_per_hun.plot.bar()